In [ ]:
#!pip install gdal==3.11.3

In [33]:
#jsonfg_path = "/Users/joana/git/gmcp/gmcp/airports.json"
#jsonfg_path = "/Users/joana/git/gmcp/gmcp/cologne-cathedral.json"
jsonfg_path = "/Users/joana/git/gmcp/gmcp/bike_lane.jsonfg"

In [34]:
# Validate geometries using GDAL

In [35]:
import sys
from osgeo import gdal, ogr

gdal.UseExceptions()

#print(f"GDAL Version: {gdal.__version__}")

In [36]:
def validate_geometries(file_path):
    print(f"Reading file: {file_path}")
    
    try:
        # Open the dataset using gdal.OpenEx (more robust than ogr.Open)
        ds = gdal.OpenEx(file_path, gdal.OF_VECTOR)
        
        if ds is None:
            print("Failed to open file. Please check the path and file integrity.")
            return


        layer = ds.GetLayer()
        layer_name = layer.GetName()
        feature_count = layer.GetFeatureCount()
        
        print(f"Layer '{layer_name}' loaded successfully.")
        print(f"Total features: {feature_count}")

        invalid_count = 0
        
        # Iterate over features
        # Note: We use a loop that ensures the feature stays in scope while we check it
        for feature in layer:
            fid = feature.GetFID()
            geom = feature.GetGeometryRef()
            
            # Check if geometry exists (some features might be null geometry)
            if geom is None:
                print(f"Feature ID {fid}: No Geometry found (NULL).")
                invalid_count += 1
                continue
                
            # Check validity
            if not geom.IsValid():
                # In standard OGR Python, IsValid() returns False but doesn't always 
                # give the specific reason string easily.
                print(f"Feature ID {fid}: INVALID geometry.")
                invalid_count += 1
        
        if invalid_count == 0:
            print("Validation Complete: All geometries are valid.")
        else:
            print(f"Validation Complete: Found {invalid_count} invalid features.")

    except Exception as e:
        print(f"An error occurred: {e}")

In [37]:
validate_geometries(jsonfg_path)

Reading file: /Users/joana/git/gmcp/gmcp/bike_lane.jsonfg
Layer 'Bicycle facility' loaded successfully.
Total features: 8
Validation Complete: All geometries are valid.


In [38]:
# Validate against JSON schema

In [39]:
#!pip install jsonschema

In [40]:
import json
from jsonschema import validate, ValidationError

In [41]:
schema_path = '/Users/joana/git/gmcp/gmcp/bike-schema.json'

In [42]:
# Load file

In [43]:
print(f"Reading file: {jsonfg_path}")
try:
    with open(jsonfg_path, 'r') as f:
        jsonfg_data = json.load(f)

except json.JSONDecodeError as e:
    print("CRITICAL ERROR: The file is not valid JSON syntax.")
    print(f"Details: {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Reading file: /Users/joana/git/gmcp/gmcp/bike_lane.jsonfg


In [44]:
try:
    with open(schema_path, 'r') as f:
        schema_data = json.load(f)
    print(f"Loaded schema: {schema_path}")
except FileNotFoundError:
    print(f"Error: Could not find {schema_path}")

Loaded schema: /Users/joana/git/gmcp/gmcp/bike-schema.json


In [45]:
try:
    # This function compares the data against the schema
    validate(instance=jsonfg_data, schema=schema_data)
    print("SUCCESS: The JSONFG file is valid according to the schema.")
    
except ValidationError as e:
    print("FAILURE: The JSONFG file is INVALID.")
    print(f"Error message: {e.message}")
    print(f"Error location: {list(e.path)}")
    
except NameError:
    print("Files were not loaded correctly, cannot validate.")

SUCCESS: The JSONFG file is valid according to the schema.
